<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# Project 1: Standardized Test Analysis

--- 
# Part 1
---

## Data

### Original Data

The datasets included in the [`data/original`](./data/original) folder.

| **Dataset** | **Description** |
| --------------- | ----------------- |
|[`act_2017.csv`](./data/original/act_2017.csv)| 2017 ACT Scores by State|
|[`act_2018.csv`](./data/original/act_2018.csv)| 2018 ACT Scores by State|
|[`act_2019.csv`](./data/original/act_2019.csv) | 2019 ACT Scores by State|
|[`sat_2017.csv`](./data/original/sat_2017.csv)| 2017 SAT Scores by State|
|[`sat_2018.csv`](./data/original/sat_2018.csv)| 2018 SAT Scores by State|
|[`sat_2019.csv`](./data/original/sat_2019.csv) | 2019 SAT Scores by State|
|[`sat_act_by_college.csv`](./data/original/sat_act_by_college.csv)| Ranges of Accepted ACT & SAT Student Scores by Colleges|

### Cleaned Data

In this notebook, ACT & SAT scores by state for the given years are concatenated to a single table per each test.

- SAT and ACT for years 2017, 2018, 2019
    - Sample output:
    
    
| |year|state|sat_participation|sat_ebrw|sat_math|sat_total|act_participation|act_english|act_math|act_reading|act_science|act_composite|
|--------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|
|0|2017|Alabama|0.05|593|572|1165|1.00|18.9|18.4|19.7|19.4|19.2|
|1|2017|Alaska|0.38|547|533|1080|0.65|18.7|19.8|20.4|19.9|19.8|
|2|2017|Arizona|0.30|563|553|1116|0.62|18.6|19.8|20.1|19.8|19.7|




|Feature            |Type     | Dataset   | Description |
|---                | ------  | --------  | ----------  |  
|year               | object  | SAT & ACT | Year        |
|state              | object  | SAT & ACT | The state where the test was taken. |
|sat_participation  | float   | SAT       | Participation rate |
|sat_ebrw           | int     | SAT       | Math and Evidence-Based Reading and Writing |
|sat_math           | int     | SAT       | Math score |
|sat_total          | int     | SAT       | Total score |
|act_participation  | float   | ACT       | Participation rate |
|act_english        | float   | ACT       | English score|
|act_math           | float   | ACT       | Math score |
|act_reading        | float   | ACT       | Reading score |
|act_science        | float   | ACT       | Science score |
|act_composite      | float   | ACT       | Composite score|
|||*number of entries*: 153  ||
|||*memory usage*: 14.5+ KB      ||





| |Column|Dtype|Description|
|---|------  |-----  |---------- |
| 0   |state|object||
| 1   |participation  |object||
| 2   |ebrw           |int64 ||
| 3   |math           |int64 ||
| 4   |total          |int64 ||
| 5   |year           |object||
| 6   |test_type      |object||
|number of entries| 155      |||
|memory usage| 9.7+ KB      |||

- ACT


| |Column|Dtype|Description|
|---|------  |-----  |---------- |
| 0  | state          |object ||
| 1  | participation  |float64||
| 2  | english        |float64||
| 3  | math           |float64||
| 4  | reading        |float64||
| 5  | science        |float64||
| 6   |composite      |float64||
| 7  | year           |object ||
| 8   |test_type      |object ||
|number of entries| 156 (includes National)|||
|memory usage| 12.2+ KB      |||


### Map Data

- Geographic map data for states can be downloaded from this [source](https://www.arcgis.com/home/item.html?id=b07a9393ecbd430795a6f6218443dccc).

## Sources

- I consulted these two medium articles to figure out the setup for using geoPanda: [article 1](https://medium.com/@minaienick/why-you-should-be-using-geopandas-to-visualize-data-on-maps-aka-geo-visualization-fd1e3b6211b4), [article 2](https://medium.com/@erikgreenj/mapping-us-states-with-geopandas-made-simple-d7b6e66fa20d)


In [1]:
%%writefile requirements0.txt
beautifulsoup4
requests
lxml
geopy
html5lib
geopandas
mapclassify

Overwriting requirements0.txt


In [2]:
import sys
from pathlib import Path

# In notebook
root_dir = Path().resolve().parents[1]  # Adjust if needed
sys.path.append(str(root_dir))

from utils import *
create_requirements_with_versions(quiet=True, upgrade=False)

In [3]:
# imports
import re
import os
import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
import requests

In [4]:
# introduce environment values

original_data_path = '../data/original'
cleaned_data_path = '../data/cleaned'

In [5]:
# available dictionary for the functions to use

us_state_abbrev = {'Alabama': 'AL', 'Alaska': 'AK', 'American Samoa': 'AS', 'Arizona': 'AZ', 'Arkansas': 'AR', 
                   'California': 'CA', 'Colorado': 'CO', 'Connecticut': 'CT', 'Delaware': 'DE', 
                   'District of Columbia': 'DC', 'Florida': 'FL', 'Georgia': 'GA', 'Guam': 'GU', 
                   'Hawaii': 'HI', 'Idaho': 'ID', 'Illinois': 'IL', 'Indiana': 'IN', 'Iowa': 'IA', 
                   'Kansas': 'KS', 'Kentucky': 'KY', 'Louisiana': 'LA', 'Maine': 'ME', 'Maryland': 
                   'MD', 'Massachusetts': 'MA', 'Michigan': 'MI', 'Minnesota': 'MN', 'Mississippi': 
                   'MS', 'Missouri': 'MO', 'Montana': 'MT', 'Nebraska': 'NE', 'Nevada': 'NV', 
                   'New Hampshire': 'NH', 'New Jersey': 'NJ', 'New Mexico': 'NM', 
                   'New York': 'NY', 'North Carolina': 'NC', 'North Dakota': 'ND', 
                   'Northern Mariana Islands': 'MP', 'Ohio': 'OH', 'Oklahoma': 'OK', 
                   'Oregon': 'OR', 'Pennsylvania': 'PA', 'Puerto Rico': 'PR', 'Rhode Island': 'RI', 
                   'South Carolina': 'SC', 'South Dakota': 'SD', 'Tennessee': 'TN', 'Texas': 'TX', 
                   'Utah': 'UT', 'Vermont': 'VT', 'Virgin Islands': 'VI', 'Virginia': 'VA', 
                   'Washington': 'WA', 'West Virginia': 'WV', 'Wisconsin': 'WI', 'Wyoming': 'WY', 'Washington D.C.': 'DC'}


colleges_states = {'Thomas Aquinas College': 'California',
 'United States Merchant Maritime Academy': 'New York',
 'United States Coast Guard Academy': 'Connecticut',
 'Washington College': 'District of Columbia',
'Southwestern University': 'Texas',
 'Berry College': 'Georgia',
 'Knox College': 'Illinois',
 'Montclair State': 'New Jersey',
 'East Carolina': 'North Carolina',
 'Siena College': 'New York', 
 'Portland State': 'Oregon',
 'Gallaudet University': 'District of Columbia'}

mandatory_sat_states = ['Colorado', 'Connecticut', 'Delaware', 'Illinois', 'Michigan', 'New Hampshire', 'Rhode Island', 'West Virginia']

mandatory_act_states = ['Hawaii', 'Kentucky', 'Louisiana', 'Nebraska ', 'North Carolina ', 'North Dakota ', 'Ohio', 'Oklahoma', 'Tennessee', 'Wyoming']



In [6]:
## helper lambda fuctions

# checks to see if a given value is a string
is_str = lambda input_value: type(input_value) == str 

# checks to see if a given string is a percentage
is_percentage = lambda given_str: given_str.strip().endswith('%')

# check to see if a given string includes integers or floats
is_numerical = lambda given_str: given_str.replace('.', '').isdigit()

# extract the year from a given string if it exists in a list
get_year_from_str = lambda given_str: re.findall('(\d{4})', given_str)

get_list_of_csv_tables = lambda path: [f for f in os.listdir(path) if f.endswith('.csv') ]


In [7]:
# helper functions


def convert_percentage_float(pctg_str):
    """
    Returns converted value of a percentage string to float.
    
    Parameters:
        pctg_str (str): The string which to be converted.
        
    Returns:
        pctg_float (float): The float value corresponding to the given input.
    """
    
    if not is_str(pctg_str):
        return pctg_str
    
    if not is_percentage(pctg_str):
        return pctg_str
    
    pctg = pctg_str.strip().strip('%')
    
    if not is_numerical(pctg):
        return pctg_str
        
    pctg_float = float(pctg)/100
    return pctg_float



def convert_camel_to_snake_case(text):
    """
    The regular expression snippet is take from 
    https://stackoverflow.com/questions/1175208/elegant-python-function-to-convert-camelcase-to-snake-case
    with slight modification.
    Parameters:
        text (str): The string input in CamelCase.
        
    Returns:
        out_text (str): The converted string in snake_case.
    """
    pattern = re.compile('((?<=[a-z0-9])[A-Z]|(?!^)[A-Z](?=[a-z]))')
    out_text = pattern.sub(r'_\1', text.replace(' ', '_')).lower().replace('__', '_')
    return out_text


In [107]:
def get_top_100_university_ratings_in_states(path=cleaned_data_path):
    """
    The code to use pd.read_html() is taken with modification from 
    # https://stackoverflow.com/questions/43590153/http-error-403-forbidden-when-reading-html
    """
    url = "https://www.topuniversities.com/where-to-study/north-america/united-states/ranked-top-100-us-universities"

    header = {
      "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
      "X-Requested-With": "XMLHttpRequest"
    }

    r = requests.get(url, headers=header)

    try:
        df1 = pd.read_html(r.text, skiprows=1,header=0)[0]
    except Exception as e:
        print(e)
        # Collect data
        soup = BeautifulSoup(r.text, "html.parser")
        rows = soup.select(".ranking-data-row")
        data = []
        
        for row in rows:
            soup = BeautifulSoup(r.text, "lxml")
            rank = row.select_one(".rank")
            name = row.select_one(".uni_name a")
            location = row.select_one(".location")
        
            if rank and name and location:
                data.append({
                    "Rank": rank.contents[0].strip().strip('='),
                    "University": name.text.strip(),
                    "Location": location.text.strip()
                })
        
        # Convert to DataFrame
        df1 = pd.DataFrame(data)
    # df1['Rank'] = df1['Rank'].str.lstrip().str.strip('=').astype(int)
    
    df1['University'].replace({
        'Stony Brook University': 'Stony Brook University SUNY',
        'Harvard University': 'Harvard College',
        'University of Wisconsin-Madison': 'University of Wisconsin, Madison',
        'North Carolina State University': 'North Carolina State University, Raleigh',
        'Indiana University Bloomington': 'Indiana University, Bloomington',
        'University of Michigan': 'University of Michigan, Ann Arbor',
        "University of Hawai'i at Manoa": 'University of Hawaii at Manoa',
    }, inplace=True)
    
    df1.to_csv(f'{path}/top_universities_rankings.csv', index = False)

    url2 = "https://www.topuniversities.com/university-rankings-articles/world-university-rankings/top-us-universities-state"
    
    r2 = requests.get(url2, headers=header)
    soup = BeautifulSoup(r2.text, 'html.parser')
    #target = soup.find('h2', text='Top US universities by state')
    target = soup.find("h2", id="art_heading_10")
    texts = [sib.text.strip('\xa0').strip() for sib in target.find_next_siblings()][:-4]
    # states = [a.split(' – ')[0] for i, a in enumerate(texts) if i%2 == 0]
    all_universities = [a for i, a in enumerate(texts) if i%2 == 1]
    print(all_universities)
    # out = [{'state': state, 'university': university.strip()} 
    #        for state, universities in zip(states, all_universities)
    #        for university in universities.split(',')
    #       ]
    sus = [su.replace(' -', '-').replace('- ', '-').replace(':\xa0', '-').split('-') for su in all_universities]
    out = [{'state': su[0], 'university': su[1]} 
           for su in sus
          ]
    print(out)
    df2 = pd.DataFrame(out)
    
    df2['university'].replace({
        'Los Angeles (UCLA)': 'University of California, Los Angeles (UCLA)',
        'Berkeley (UCB)': 'University of California Berkeley (UCB)',
        'San Diego (UCSD)': 'University of California, San Diego (UCSD)',
        'Davis (UCD)': 'University of California, Davis (UCD)',
        'Santa Cruz (UCSC)': 'University of California, Santa Cruz (UCSC)',
        'Riverside (UCR)': 'University of California, Riverside (UCR)',
        'Irvine (UCI)': 'University of California, Irvine (UCI)',
        'Santa Barbara (UCSB)': 'University of California, Santa Barbara (UCSB)',
        'Harvard University': 'Harvard College',
        "University of Hawai'i at Manoa": 'University of Hawaii at Manoa',
    }, inplace=True)
    
    df2.to_csv(f'{path}/states_universities.csv', index = False)


In [108]:
def get_coordinates_for_universities(list_of_university_names, path=cleaned_data_path):
    """
    # https://towardsdatascience.com/geoparsing-with-python-c8f4c9f78940
    Returns a dataframe with the list of universities and their coordinates
    """
    
    from geopy.geocoders import Nominatim
    from geopy.exc import GeocoderTimedOut
    geolocator = Nominatim(timeout=10, user_agent="http")

    out = {}

    for school in list_of_university_names: 
        try:
            location = geolocator.geocode(school)
            if not location:
                location = geolocator.geocode(school.split('(')[0])
            if not location:
                location = geolocator.geocode(school.rstrip(' SUNY'))
            if not location:
                location = geolocator.geocode(school.split(', ')[0])
            if location:
                out.update({school: location})
                if location.raw['type'] not in ['university', 'college', 'school', 'museum']:
                    print(school, location.raw['type'], sep=', ')
        except GeocoderTimedOut as e:
            print("Error: geocode failed on input %s with message %s"%(school, e))

    df = pd.DataFrame(out).T
    df.reset_index(inplace=True)
    df.rename(columns = {'index': 'school',0: 'place', 1: 'coordinates'}, inplace=True)
    df['state'] = df['place'].str.split(', ').str[-3]
    df.loc[~df['state'].isin(us_state_abbrev.keys()), 'state'] = df['place'].str.split(', ').str[-2]
    df = df[df['state'].isin(us_state_abbrev.keys())]
    df.to_csv(f'{path}/university_coordinates_states.csv', index = False)

In [109]:
def clean_data_frame(df, pctg_columns = [], col_rename_dict = {}):
    """
    Returns the modified version of the given dataframe after changing column names 
    and other required conversions.
    
    Parameters:
        df (DataFrame): The input data frame
        pctg_columns (list): A list of that include column names with percentage values.
        col_rename_dict (dict): A dictionary including the original column names 
        and their correpsonding new names.
        
    Returns:
        df (DataFrame): The modified/cleaned dataframe.
    
    """
    
    # drop columns with all nan entries
    df.dropna(how='all', axis = 1, inplace = True)
    
    # drop rows with all nan entries
    df.dropna(how='all', axis = 0, inplace = True)
    
    # convert column names to desired names 
    df.rename(columns = col_rename_dict, inplace=True)
    
    # convert percentage values to floats
    for pctg_col in pctg_columns:
        if pctg_col in df.columns:
            df[pctg_col] = df[pctg_col].map(convert_percentage_float)
            
    df = df.rename(columns = lambda x: convert_camel_to_snake_case(x))
                
    return df
    
    
def base_read_clean_csv(path, fname, converters = {}, dtypes = {}, pctg_columns =  [], col_rename_dict = {}):
    """
    Reads a data table and does some data cleaning and column name conversions and 
    returns a pandas dataframe.
    
    Parameters:
        path (str): The path to the directory including the tables.
        fname (str): The .csv (or other formats) filename.
        converters (dict): A dictionary used for possible column value conversions.
        dtypes (dict): A dictionary used for possible casting of column types.
        pctg_columns (list): A list of that include column names with percentage values.
        col_rename_dict (dict): A dictionary including the original column names 
        and their correpsonding new names.
        
    Returns:
        df (DataFrame): The dataframe read and cleaned for the given file.
    
    """
    df = pd.read_csv(f'{path}/{fname}', 
                        na_values=['*', 'N/A', '<NA>'],
                         converters = converters,
                         dtype = dtypes,
                         keep_default_na = True,
                       )

    df = clean_data_frame(df, pctg_columns = pctg_columns, col_rename_dict = col_rename_dict)
    return df



def read_data_write_clean_data(path=original_data_path, write_path=cleaned_data_path):
    """
    Reads all the datasets in the form of DataFrame for all the 
    tables in the original data directory and conducts data cleaning on each
    and saves them in the write_path.
    It also combines all ACT and SAT scores for 3 years to their respective dataframes.
    
    Parameters:
        path (str): The path to the directory of the original datasets.
        write_path (str): The path to the directory of the clean datasets.
        
    Returns:
        result (dict): The dictionary output of all the datasets.
    """
    

    
    pctg_columns = ['Participation', 'Accept Rate', 'Percent']
    col_rename_dict = {'Evidence-Based Reading and Writing': 'EBRW',
                         'Participation Rate': 'Participation'}
    
    file_names = get_list_of_csv_tables(path)
    
    for fname in file_names:
        name = fname.split('.csv')[0]
        converters = {}
        
        dtypes = {k: str for k in ['CDS', 'CCode', 'SCode', 'CDCode']} # for California data
        
        if name in ['act_2017']:
            converters = {'Composite': lambda s: float(s.strip('x'))}
            
        if name.endswith('by_intended_college_major'):
            name = name.replace('by_intended_college_major', 'by_major')
            
        df = base_read_clean_csv(path, fname, converters, dtypes, pctg_columns, col_rename_dict)
        
        years = get_year_from_str(name)
        if years:
            df['year'] = years[0]
            
        if name.endswith('_ca'):
            df['state'] = 'California'
            
        out_file_name = f'{write_path}/{name}.csv'
        
        if name == 'sat_act_by_college':
            df['school'].replace('--', ', ', inplace=True, regex=True)
            df['school'].replace('—​', ', ', inplace=True, regex=True)
        
            # clean school names to be merged with rankings later
            df['school'].replace({
            'University of California, Los Angeles': 'University of California, Los Angeles (UCLA)',
            'University of California, Berkeley': 'University of California Berkeley (UCB)',
            'University of California, San Diego': 'University of California, San Diego (UCSD)',
            'University of California, Davis': 'University of California, Davis (UCD)',
            'University of California, Santa Cruz': 'University of California, Santa Cruz (UCSC)',
            'University of California Riverside': 'University of California, Riverside (UCR)',
            'University of California, Irvine': 'University of California, Irvine (UCI)',
            'University of California, Santa Barbara': 'University of California, Santa Barbara (UCSB)', 
            'New York University': 'New York University (NYU)',
            'Massachusetts Institute of Technology': 'Massachusetts Institute of Technology (MIT)',
            'Georgia Institute of Technology': 'Georgia Institute of Technology (Georgia Tech)',
            'California Institute of Technology': 'California Institute of Technology (Caltech)',
            'New Jersey Institute of Technology': 'New Jersey Institute of Technology (NJIT)',
            'University of Illinois, Chicago': 'University of Illinois, Chicago (UIC)',
            'University at Albany, SUNY': 'University at Albany SUNY',
            'Stony Brook University, SUNY': 'Stony Brook University SUNY',
            'University at Buffalo, SUNY': 'University at Buffalo SUNY',
            'University of Oklahoma': 'University of Oklahoma, Norman',
            'University of Colorado, Boulder': 'University of Colorado Boulder',
            'Virginia Polytechnic Institute and State University': 'Virginia Polytechnic Institute (Virginia Tech)',
            'University of Texas, Austin': 'University of Texas at Austin',
            'Texas A&M University, College Station': 'Texas A&M University',
            'University of Texas, Dallas': 'University of Texas Dallas',
            'University of Illinois, Urbana-​Champaign': 'University of Illinois at Urbana-Champaign',
            'Rutgers The State University of New Jersey, Newark': 'Rutgers - The State University of New Jersey, Newark', 
            'Rutgers The State University of New Jersey, New Brunswick': 'Rutgers - The State University of New Jersey, New Brunswick', 
            'Purdue University, West Lafayette': 'Purdue University',
            }, inplace=True)
        
        df.to_csv(out_file_name, index = False)
        
    sat_and_act = []
    for test_type in ['sat', 'act']:
        combined_data = pd.concat([pd.read_csv(f'{write_path}/{test_type}_{year}.csv') for year in [2017, 2018, 2019]])
        sat_and_act.append(combined_data.set_index(['year', 'state']).add_prefix(f'{test_type}_').reset_index())
        combined_data.to_csv(f'{write_path}/{test_type}.csv', index = False)
        
    sat_and_act_df = sat_and_act[0].merge(sat_and_act[1])

    sat_and_act_df['sat_mandatory'] = False
    sat_and_act_df.loc[sat_and_act_df['state'].isin(mandatory_sat_states), 'sat_mandatory']=True
    sat_and_act_df['act_mandatory'] = False
    sat_and_act_df.loc[sat_and_act_df['state'].isin(mandatory_act_states), 'act_mandatory']=True
    
    sat_and_act_df.to_csv(f'{write_path}/act_and_sat.csv', index = False)
    
def combine_sat_act_by_college_with_states(path=cleaned_data_path):
    # import two dataframes to merge them
    df1 = pd.read_csv(f'{path}/sat_act_by_college.csv'); # shape: (416, 8)
    df2 = pd.read_csv(f'{path}/states_universities.csv');# (385, 2)
    df3 = pd.read_csv(f'{path}/top_universities_rankings.csv'); 
    university_coordinates = pd.read_csv(f'{cleaned_data_path}/university_coordinates_states.csv')
    
    # merge two dataframes to add state as a column
    df = df1.merge(df2, how='left', left_on='school', right_on='university').drop(columns='university'); # shape (417, 9)
    
    

    df = df.merge(university_coordinates, right_on='school', left_on='school', how='left')
    df['state_x'].fillna(df['state_y'], inplace=True)
    df['state'] = df['state_x']
    df['state'].fillna(df['school'].map(colleges_states), inplace=True)
    
    conv = {}
    for state, abbrv in us_state_abbrev.items():
        for school in df[df['state'].isnull()]['school']:
            if state in school or f'{abbrv}' in school:
                conv.update({school: state}) 

    df['state'].fillna(df['school'].map(conv), inplace=True)
    
    df = df.replace('--', '-')
    df[['sat_25th_percentile', 'sat_75th_percentile']] = df['sat_total_25th-75th_percentile'].str.strip().str.split().str[-1].str.split('-', expand=True, n=1)
    df[['act_25th_percentile', 'act_75th_percentile']] = df['act_total_25th-75th_percentile'].str.strip().str.split().str[-1].str.split('-', expand=True, n=1)
    df.replace('', np.nan, inplace=True) 
    df = df.astype({c: float for c in ['act_25th_percentile', 'act_75th_percentile', 'sat_25th_percentile', 'sat_75th_percentile']})
    df.drop(columns=['sat_total_25th-75th_percentile', 'act_total_25th-75th_percentile'], inplace=True)
    df['sat_median'] = df[['sat_25th_percentile','sat_75th_percentile']].mean(axis = 1)
    df['act_median'] = df[['act_25th_percentile','act_75th_percentile']].mean(axis = 1)
    
    df.dropna(subset=['sat_median'], inplace=True)
    df['sat_median'] = df['sat_median'].astype(int)

    
    df['test_blind?'] = df['test_optional?']=='Yes (TB)'
    df['test_free?'] = df['test_optional?']=='Yes (TF)'
    df['test_optional?'].replace({k: 'Yes' for k in ['Yes*', 'Yes (TB)', 'Yes (TF)']}, inplace=True)
    df['test_optional?'].replace({'Yes': True, 'No': False}, inplace = True)
    
    df['sat_mandatory'] = False
    df.loc[df['state'].isin(mandatory_sat_states), 'sat_mandatory']=True
    df['act_mandatory'] = False
    df.loc[df['state'].isin(mandatory_act_states), 'act_mandatory']=True
    
    df = df.merge(df3, how='left', left_on='school', right_on='University').drop(columns='University'); 

    df['Rank'] = df['Rank'].astype('Int64')
    df.rename(columns = {'sat_25th_percentile': 'sat_min', 'act_25th_percentile': 'act_min'}, inplace=True)
              
              
    df = df[['school', 'state', 'number_of_applicants', 'accept_rate', 'sat_min', 'sat_median', 'act_min', 'act_median', 'test_optional?', 
             'Rank', 'coordinates', 'sat_mandatory', 'act_mandatory']]
    
    
    df.to_csv(f'{path}/sat_act_by_college_in_states.csv', index = False)


In [110]:
def save_high_ranking_colleges_with_coordinates():
    """
    Returns a limited number of rows for the high ranking colleges.
    """
    df = pd.read_csv(f'{cleaned_data_path}/sat_act_by_college_in_states.csv')
    df = df[~df['Rank'].isnull()]
    df.sort_values(by='Rank', ascending=True, inplace=True)
    df['Rank'] = df['Rank'].astype(int)
    df.set_index('Rank', inplace=True)
    df.reset_index(inplace=True)
    df.to_csv(f'{cleaned_data_path}/high_ranking_colleges_with_coordinates.csv', index=False)
    

In [111]:
def combine_universities_required_scores_with_average_state_scores(path=cleaned_data_path):
    #median_acceptance_score_by_average_of_universities_in_states 
    college_req = pd.read_csv(f'{path}/sat_act_by_college_in_states.csv')
    college_req_scores = college_req.groupby('state')[['sat_min', 'sat_median', 'act_min', 'act_median']].mean().reset_index()

    act_and_sat_all_years = pd.read_csv(f'{path}/act_and_sat.csv')
    #act_and_sat = act_and_sat_all_years[act_and_sat_all_years['year']==2019][['sat_total', 'act_composite', 'state']]
    act_and_sat = act_and_sat_all_years.set_index('year').groupby('state').max().reset_index()[['sat_total', 'act_composite', 'state']]

    df = act_and_sat.merge(college_req_scores, on='state', how='left')
    df['sat_mandatory'] = False
    df.loc[df['state'].isin(mandatory_sat_states), 'sat_mandatory']=True
    df['act_mandatory'] = False
    df.loc[df['state'].isin(mandatory_act_states), 'act_mandatory']=True
    df.rename(columns = {'act_composite': 'act_total'}, inplace=True)
    df = df[['state', 'sat_min', 'sat_median', 'sat_total', 'act_min', 'act_median', 'act_total', 'sat_mandatory', 'act_mandatory']]
    df.to_csv(f'{path}/req_scores_state_averages.csv', index = False)

In [112]:
def get_selected_data(path=cleaned_data_path):
    """
    Returns a dictionary of all the selected datasets.
    
    Parameters:
        path (str): The path to the directory of the original datasets.
        
    Returns:
        result (dict): The dictionary output of all the datasets.
    """
    out = {}
    
    selected_datasets = ['sat_act_by_college_in_states.csv', 'sat_2019_by_major.csv', 'act_and_sat.csv', 'high_ranking_colleges_with_coordinates.csv', 'req_scores_state_averages.csv']
    
    for name in selected_datasets:
        out.update({ name.split('.')[0] : pd.read_csv(f'{path}/{name}', dtype={'year': str, 
#                                                                                'Rank': 'Int64'
                                                                              })})

    return out


In [113]:
## code to run for generating the table for the coordinates of colleges/universities

list_of_schools = pd.read_csv(f'{cleaned_data_path}/sat_act_by_college.csv')['school'].tolist()
#get_coordinates_for_universities(list_of_schools)

In [114]:
get_top_100_university_ratings_in_states()

No tables found
['Kentucky - University of Kentucky', 'Louisiana - Tulane University', 'Maryland - Johns Hopkins University', 'Massachusetts - Massachusetts Institute of Technology (MIT)', 'Michigan - University of Michigan-Ann Arbor', 'Minnesota - University of Minnesota Twin Cities', 'Mississippi - Mississippi State University 1001-1200', 'Missouri - Washington University in St. Louis', 'Montana - University of Montana Missoula', 'Nebraska - University of Nebraska – Lincoln', 'New Hampshire:\xa0Dartmouth College', 'New Jersey- Princeton University', 'New Mexico - University of New Mexico']
[{'state': 'Kentucky', 'university': 'University of Kentucky'}, {'state': 'Louisiana', 'university': 'Tulane University'}, {'state': 'Maryland', 'university': 'Johns Hopkins University'}, {'state': 'Massachusetts', 'university': 'Massachusetts Institute of Technology (MIT)'}, {'state': 'Michigan', 'university': 'University of Michigan'}, {'state': 'Minnesota', 'university': 'University of Minnesota

In [ ]:
read_data_write_clean_data()
combine_sat_act_by_college_with_states()
save_high_ranking_colleges_with_coordinates()
combine_universities_required_scores_with_average_state_scores()